<a href="https://colab.research.google.com/github/ShoYanase/drs-prototype/blob/master/drs_prot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [0]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import re
import os
import tensorflow as tf
import pprint

import MeCab

from gensim.models import Word2Vec
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm, decomposition
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from joblib import dump,load

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Programs

## Parsing Input

In [0]:
#文章を文単位で分割
def parse_Paragraph(paragraph):
  sentences = re.split('. |。|．|\.', paragraph)
  sentences = [re.sub(',|，', '、',s) for s in sentences]
  res = [s+'。' for s in sentences][:-1]
  return res

In [0]:
#mecabで文を分かち書き
def parse_Sentence(sentence):
  m = MeCab.Tagger("-Ochasen")
  parsed = m.parse(sentence)
  return parsed

In [0]:
#含まれる文を全て分かち書きした文章
def parse_Parag_to_Word(paragraph):
  sentences = parse_Paragraph(paragraph)
  word_parsed_paragraph = [[p.split('\t') for p in parse_Sentence(s).split('\n')[:-2]] for s in sentences]
  return word_parsed_paragraph

In [0]:
print(parse_Parag_to_Word("図26、図27より、バッファサイズが大きくなる程実行時間が短くなることがわかる.さらにそれぞれの近似曲線より、バッファサイズをx、実行時間をyとすると、両者の関係はy=a/x(aはある特定の定数)という方程式で近似的に表せることができ、実行時間はバッファサイズに反比例していることがわかる.また図27より、read、writeによる実装よりも、fread、fwriteによる実装の方が実行時間が速いことがわかる.実行時間がバッファサイズに反比例したのは、次のような理由が考えられる.cでは、ファイルの内容がバッファサイズごとに読み取り・書き込みが行われる.その処理の回数はバッファサイズに反比例する.よって読み取り・書き込みの回数が増えることによって実行時間が増えると考えられる.read、writeによる実装よりも、fread、fwriteによる実装の方が実行時間が速いのには、以下のような理由が考えられる.cは、両者とも同じバッファサイズで実行しているが、システムコールread、write関数の呼び出し回数が大きく違う.これによって両者の実行時間の違いが出ていると考えられる."))

[[['図', 'ズ', '図', '名詞-一般', '', ''], ['26', '26', '26', '名詞-数', '', ''], ['、', '、', '、', '記号-読点', '', ''], ['図', 'ズ', '図', '名詞-一般', '', ''], ['27', '27', '27', '名詞-数', '', ''], ['より', 'ヨリ', 'より', '助詞-格助詞-一般', '', ''], ['、', '、', '、', '記号-読点', '', ''], ['バッファ', 'バッファ', 'バッファ', '名詞-一般', '', ''], ['サイズ', 'サイズ', 'サイズ', '名詞-一般', '', ''], ['が', 'ガ', 'が', '助詞-格助詞-一般', '', ''], ['大きく', 'オオキク', '大きい', '形容詞-自立', '形容詞・イ段', '連用テ接続'], ['なる', 'ナル', 'なる', '動詞-自立', '五段・ラ行', '基本形'], ['程', 'ホド', '程', '名詞-非自立-一般', '', ''], ['実行', 'ジッコウ', '実行', '名詞-サ変接続', '', ''], ['時間', 'ジカン', '時間', '名詞-副詞可能', '', ''], ['が', 'ガ', 'が', '助詞-格助詞-一般', '', ''], ['短く', 'ミジカク', '短い', '形容詞-自立', '形容詞・アウオ段', '連用テ接続'], ['なる', 'ナル', 'なる', '動詞-自立', '五段・ラ行', '基本形'], ['こと', 'コト', 'こと', '名詞-非自立-一般', '', ''], ['が', 'ガ', 'が', '助詞-格助詞-一般', '', ''], ['わかる', 'ワカル', 'わかる', '動詞-自立', '五段・ラ行', '基本形'], ['。', '。', '。', '記号-句点', '', '']], [['さらに', 'サラニ', 'さらに', '副詞-助詞類接続', '', ''], ['それぞれ', 'ソレゾレ', 'それぞれ', '名詞-副詞可能', '', ''], ['の', 'ノ', 'の', '助詞-連体化

## 要素分類(import Parsing Input)

Content_Classification(paragraph): 引数に文章を入れて分類結果リストを得る

### DataPath

In [0]:
#点数データのパス
path_Content_points = "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Content_points.csv"
#要素分類モデル
path_sentence_attr_model = "gdrive/My Drive/Colab Notebooks/研究/data/Statement_attributes/model_svc.model"
#w2vモデル
path_wv_model = "gdrive/My Drive/Colab Notebooks/研究/data/word2vec.model"

In [0]:
df_Content_points = pd.read_csv(path_Content_points)
df_Content_points.head()

,from,to,point
0,claim,claim,5
1,claim,ground,0
2,ground,claim,7
3,ground,ground,5


### Load Model

In [0]:
#要素分類モデル読み込み
sentence_attr_model = load(path_sentence_attr_model)
pprint.pprint(SVC.get_params(sentence_attr_model,deep=True))
#w2vモデル読み込み
wv_model = Word2Vec.load(path_wv_model)

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': True,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### 主成分分析

In [0]:
def pca_vec(vec):
  vec_ = np.array([vec[:,n] for n in range(len(vec[0])-1)])
  pca = decomposition.PCA(n_components=1)
  vec_transformed = pca.fit_transform(vec_)
  return vec_transformed[:,0]

### mecabの分かち書きからベクトル生成

In [0]:
def make_vec(sentence, wv_model):
  vec = np.array([wv_model.wv[w] for w in [sentence[n][0] for n in range(len(sentence)-1)]])
  return pca_vec(vec)

### Prediction

In [0]:
#文単位で分割された文章を入力　推測の結果をリストで返す
def predict_Sentence_attr(vec, model):
  prediction = model.predict(vec).astype(int)
  print(prediction[0])
  prediction = np.where(prediction==0, 'claim', prediction)
  prediction = np.where(prediction=='-1', 'ground', prediction)
  prediction = np.where(prediction=='1', 'ground', prediction)
  prediction = np.where(prediction=='2', 'ground', prediction)
  return prediction.tolist()

In [0]:
def Content_Classification(paragraph):
  parsed = parse_Parag_to_Word(paragraph)
  arr_attr = []
  for p in parsed:
    vec = make_vec(p, wv_model)
    arr_attr.append(predict_Sentence_attr(vec.reshape(1,-1), sentence_attr_model)[0])
  return arr_attr

### Sample

In [0]:
Content_Classification("図26、図27より、バッファサイズが大きくなる程実行時間が短くなることがわかる.さらにそれぞれの近似曲線より、バッファサイズをx、実行時間をyとすると、両者の関係はy=a/x(aはある特定の定数)という方程式で近似的に表せることができ、実行時間はバッファサイズに反比例していることがわかる.また図27より、read、writeによる実装よりも、fread、fwriteによる実装の方が実行時間が速いことがわかる.実行時間がバッファサイズに反比例したのは、次のような理由が考えられる.cでは、ファイルの内容がバッファサイズごとに読み取り・書き込みが行われる.その処理の回数はバッファサイズに反比例する.よって読み取り・書き込みの回数が増えることによって実行時間が増えると考えられる.read、writeによる実装よりも、fread、fwriteによる実装の方が実行時間が速いのには、以下のような理由が考えられる.cは、両者とも同じバッファサイズで実行しているが、システムコールread、write関数の呼び出し回数が大きく違う.これによって両者の実行時間の違いが出ていると考えられる.")

1
1
1
1
1
1
0
1
1
1


['ground',
 'ground',
 'ground',
 'ground',
 'ground',
 'ground',
 'claim',
 'ground',
 'ground',
 'ground']

### 加点行列をつくる

In [0]:
def Content_addmat(arr_content, matsize, df_Content_points):
  points_mat = np.zeros((matsize, matsize), dtype=int)
  for i in range(matsize):
    for j in range(i+1, matsize):
      if i != j:
        res = df_Content_points[df_Content_points['from']==arr_content[i]]
        res = res[res['to']==arr_content[j]]
        points_mat[i][j] = res["point"]+i-j+1
        points_mat[j][i] = res["point"]+i-j+1
  return points_mat
  
Content_addmat(["ground", "ground", "ground", "ground", "claim"],5, df_Content_points)

array([[0, 5, 4, 3, 4],
       [5, 0, 5, 4, 5],
       [4, 5, 0, 5, 6],
       [3, 4, 5, 0, 7],
       [4, 5, 6, 7, 0]])

## 接続詞(import Parsing Input, 要素分類)

### Datapath

In [0]:
#接続関係と点数データのパス
path_Conjuction_attr =  "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Conjuction_attr.csv"
path_Conjuction_points =  "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Conjuction_points.csv"
#山本 和英, 齋藤 真実の分類
path_Conjuction_attr_yamamoto =  "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Yamamoto_Rules/Conjuction_attr.csv"
path_Conjuction_points_yamamoto =  "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Yamamoto_Rules/Conjuction_points.csv"

In [0]:
df_conjuction_attr = pd.read_csv(path_Conjuction_attr)
df_conjuction_points = pd.read_csv(path_Conjuction_points)

### Preview

In [0]:
df_conjuction_attr.head()

,word,attr
0,かくして,解説
1,すると,解説
2,然し乍ら,制限
3,即ち,解説
4,けれども,制限


In [0]:
df_conjuction_points.head(20)

,attr,if_loc,if_is,point,from_start,from_end,to_start,to_end
0,解説,NaN,NaN,0,NaN,-2,0,0.0
1,解説,NaN,NaN,20,0.0,0,-1,-1.0
2,対比,1.0,claim,10,-1.0,0,1,1.0
3,対比,1.0,ground,10,-1.0,0,1,1.0
4,例示,NaN,NaN,-5,NaN,-2,0,0.0
5,例示,NaN,NaN,20,0.0,0,-1,-1.0
6,付加,1.0,claim,10,-1.0,0,1,1.0
7,付加,1.0,ground,5,-1.0,-1,0,0.0
8,転換,NaN,NaN,-40,NaN,-1,0,NaN
9,制限,NaN,NaN,10,0.0,0,1,1.0


### 文頭の接続詞から属性探索

In [0]:
def search_Conjuction(df_attr, sentence):
  if '接続詞' in sentence[0][3]:
    query = 'word == \"'+sentence[0][0]+'\"'
    arr_attr = df_attr.query(query)
    print(sentence[0][0]," is ", arr_attr["attr"].tolist())
    return arr_attr
  else:
    return 0

sentence = parse_Parag_to_Word("なんだかやる気が出ないお。よって、私は飯を食い床に就く。")
search_Conjuction(df_conjuction_attr, sentence[1])

よって  is  ['解説']


,word,attr
52,よって,解説


### 属性からルール探索

In [0]:
def search_Rules(df_rules, attr):
  arr_rules = pd.DataFrame()
  if attr is not 0:
    query = 'attr == \"'+attr['attr']+'\"'
    res = df_rules.query(query)
    arr_rules = pd.concat([arr_rules,res], axis=0)
  return arr_rules  

sentence = parse_Parag_to_Word("なんだかやる気が出ないお。よって私は飯を食い床に就く。")
attrs = search_Conjuction(df_conjuction_attr, sentence[1])
for index, attr in attrs.iterrows():
#  print(index,attr)
  rules = search_Rules(df_conjuction_points, attr)
  print(rules.head())

よって  is  ['解説']
  attr  if_loc if_is  point  from_start  from_end  to_start  to_end
0   解説     NaN   NaN      0         NaN        -2         0     0.0
1   解説     NaN   NaN     20         0.0         0        -1    -1.0


### 加点行列をつくる

In [0]:
#search_ConjuctionからDataFrame.iterrows()でiterateしてattrでrsearch_Rulesから抽出適用・構造候補の加点行列を作る
def Conjuction_addmat(rules, matsize, cur, arr_attr):
  points_mat = np.zeros((matsize, matsize), dtype=int)
  rules['from_start'] = rules['from_start'].fillna(-cur)
  rules['from_end'] = rules['from_end'].fillna(-cur)
  rules['to_start'] = rules['to_start'].fillna(-cur)
  rules['to_end'] = rules['to_end'].fillna(matsize)
#  print(rules)
  for index, rule in rules.iterrows():
#    print(rule)
    point = int(rule['point'])
    adder_fromstart = cur + int(rule['from_start'])
    adder_fromend = cur + int(rule['from_end'])+1
    adder_tostart = cur + int(rule['to_start'])
    adder_toend = cur + int(rule['to_end'])+1
    
    if np.isnan(rule['if_loc']) != True:
      if arr_attr[cur+int(rule['if_loc'])] == rule['if_is']:
        print(point,"points [",adder_fromstart,":",adder_fromend,"] to [",adder_tostart,":",adder_toend,"] with \"", rules['attr'].tolist()[0],"\"")
#        print(points_mat[adder_fromstart:adder_fromend][adder_tostart:adder_toend])
        points_mat[adder_fromstart:adder_fromend,adder_tostart:adder_toend] += point
        points_mat[adder_tostart:adder_toend,adder_fromstart:adder_fromend] += point
    else:
      print(point,"points [",adder_fromstart,":",adder_fromend,"] to [",adder_tostart,":",adder_toend,"] with \"", rules['attr'].tolist()[0],"\"")
      points_mat[adder_fromstart:adder_fromend,adder_tostart:adder_toend] += point
      points_mat[adder_tostart:adder_toend,adder_fromstart:adder_fromend] += point
#    print(points_mat,"\n")
  return points_mat

Conjuction_addmat(rules, 5, 3, ["ground", "ground", "ground", "ground", "claim"])

0 points [ 0 : 2 ] to [ 3 : 4 ] with " 解説 "
20 points [ 3 : 4 ] to [ 2 : 3 ] with " 解説 "


array([[ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0, 20,  0],
       [ 0,  0, 20,  0,  0],
       [ 0,  0,  0,  0,  0]])

In [0]:
def Paragraph_to_Conjuction_addmat(df_conjuction_attr, df_conjuction_points, sentences, arr_content):
  len_sentence = len(sentences)
  addmat = np.zeros((len(sentences), len(sentences)), dtype=int)
  cases = pd.DataFrame({'matrix':[addmat], 'label':['']})

  def makemat(s, attr):
    label = str(str(s+1)+"文目:「"+attr['word']+"」が"+attr['attr']+"\n")
    #接続関係とルールのマッチング
    rules = search_Rules(df_conjuction_points, attr)
    mat = Conjuction_addmat(rules, len_sentence, s, arr_content)
    pprint.pprint(mat)
    return label, mat

  for s in range(len_sentence):
    #単語と接続関係のマッチング
#    print(sentences[s])
    arr_attr = search_Conjuction(df_conjuction_attr, sentences[s])
    if type(arr_attr) is not int:
        base0 = cases.copy()
        cases.drop(cases.index, inplace=True)
        for index, attr in arr_attr.iterrows():
            label, matrix = makemat(s, attr)    
            base1 = base0.copy()
            base1['matrix'] = base1['matrix'].map(lambda x: x + matrix)
            base1['label'] = base1['label'].map(lambda x: x + label)
            cases = pd.concat([cases, base1])
    
  return cases

## 指示語

In [0]:
def Directive_addmat(sentences, matsize, point):
  points_mat = np.zeros((matsize, matsize), dtype=int)
  for s in range(1,len(sentences)):
    if re.match("この|あの|その|これ|それ",sentences[s][0][0]):
      print(sentences[s][0][0], " in ", s, "文目")
      points_mat[s-1][s] += point
      points_mat[s][s-1] += point
  print(points_mat)
  return points_mat

## 他キーワード

In [0]:
#単語一覧のパス
path_Keywords = "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Keywords.csv"

df_keywords = pd.read_csv(path_Keywords)
df_keywords.head()

,word,point,from,to
0,同様,10,0,1
1,次,10,0,1
2,以下,10,0,1
3,さらに,10,-1,0


In [0]:
def Keyword_addmat(sentences, matsize, df_keywords):
  points_mat = np.zeros((matsize, matsize), dtype=int)
  for s in range(len(sentences)-1):
    for index, row in df_keywords.iterrows():
      keyword = row["word"]
      point = row["point"]
      for w in sentences[s]:
        if keyword in w[0]:
          print(keyword, " in ", s, "文目")
          points_mat[s+row["from"]][s+row["to"]] += point
          points_mat[s+row["to"]][s+row["from"]] += point
  print(points_mat)
  return points_mat

## 共通の単語

In [0]:
#単語一覧のパス
path_Exeptwords = "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Exeptwords.csv"

df_exeptwords = pd.read_csv(path_Exeptwords)
df_exeptwords.head()

,word
0,考え
1,考える
2,する
3,いる
4,なる


In [0]:
def Commonword_addmat(sentences, matsize, df_exeptwords, coef, thres):
  points_mat = np.zeros((matsize, matsize), dtype=int)
  per_mat = np.zeros((matsize, matsize), dtype=float)
  arr_exeptword = df_exeptwords["word"].tolist()

  def extract_words(sentence):
    s =[]
    for w in sentence:
     if re.match('名詞|動詞|形容詞|形容動詞', w[3]):
       s.append(w[0])
    sentence_words = list(set(s) - set(arr_exeptword))
    return sentence_words

  def compare_sentences(sentence1, sentence2):
    arr_commonwords = list(set(sentence1) & set(sentence2))
    meanlength = (len(sentence1) + len(sentence2)) / 2
    percentage = len(arr_commonwords) / meanlength
    ##一致率*係数=点数
    #point = int(percentage * coef)
    ##一致率>閾値=点数
    if percentage >= thres:
      point = coef
    else:
      point = 0

    return point, percentage

  for s1 in range(matsize-1):
    sentence_words1 = extract_words(sentences[s1])
    for s2 in range(s1+1, matsize):
      sentence_words2 = extract_words(sentences[s2])
      point, percentage = compare_sentences(sentence_words1, sentence_words2)
      per_mat[s1,s2] = percentage
      per_mat[s2,s1] = percentage

      if point > 0:
        print(s1+1, "文目と", s2+1, "文目の一致率: ", percentage)
        points_mat[s1,s2] = point
        points_mat[s2,s1] = point
  print(points_mat)
  print(per_mat)
  return points_mat

Commonword_addmat(parse_Parag_to_Word("グラフを見ると、バッファサイズが非常に小さいときファイルサイズが大きいと非常に時間がかかっている。またread/writeの方がrecv/sendより僅かの差だが、多くの時間を要している。そして、バッファサイズを大きくしていっても、read/writeがrecv/sendより速くなることはなかった。この要因は、ファイルの通信処理はソケットを用いて通信した方が速くなるからであると考える。"), 4, df_exeptwords, 7, 0.5)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[0.         0.0952381  0.18181818 0.        ]
 [0.0952381  0.         0.47619048 0.11111111]
 [0.18181818 0.47619048 0.         0.10526316]
 [0.         0.11111111 0.10526316 0.        ]]


array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

# main

In [0]:
#要素分類
##点数
path_Content_points = "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Content_points.csv"
df_Content_points = pd.read_csv(path_Content_points)
##要素分類モデル
path_sentence_attr_model = "gdrive/My Drive/Colab Notebooks/研究/data/Statement_attributes/model_svc.model"
##w2vモデル
path_wv_model = "gdrive/My Drive/Colab Notebooks/研究/data/word2vec.model"

#接続詞
##接続関係と点数
path_Conjuction_attr =  "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Conjuction_attr.csv"
path_Conjuction_points =  "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Conjuction_points.csv"
##山本 和英, 齋藤 真実の分類
path_Conjuction_attr_yamamoto =  "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Yamamoto_Rules/Conjuction_attr.csv"
path_Conjuction_points_yamamoto =  "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Yamamoto_Rules/Conjuction_points.csv"
df_conjuction_attr = pd.read_csv(path_Conjuction_attr)
df_conjuction_points = pd.read_csv(path_Conjuction_points)

#指示代名詞
##点数
directive_point = 10

#キーワード
##単語一覧
path_Keywords = "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Keywords.csv"
df_keywords = pd.read_csv(path_Keywords)

#共通の単語
##単語一覧
path_Exeptwords = "gdrive/My Drive/Colab Notebooks/研究/data/data_prot/Rules/Exeptwords.csv"
df_exeptwords = pd.read_csv(path_Exeptwords)
##点数
commonwords_coef = 6
##閾値
commonwords_thres = 0.5

In [0]:
#生の文章(paragraph)と結論の位置(target)から点数を出す
def Point_matrix(paragraph, target):
  #文を分かち書き
  sentences = parse_Parag_to_Word(paragraph)
  len_sentence = len(sentences)
  
  #文章の頭から指定の文まで抜き出し
  if len_sentence >= target:
    sentences = sentences[:target]
    len_sentence = len(sentences)

  #点数の行列
  points_mat = np.array([[50]*target for i in range(len_sentence)])

  #要素分類
  print("---------------要素分類------------------------------------------------")
  arr_content = Content_Classification(paragraph)[:-1]
  arr_content.append('claim')
  #arr_content = ["ground", "ground", "ground", "claim", "claim", "claim", "claim", "claim", "ground", "claim"]
  content_addmat = Content_addmat(arr_content, target, df_Content_points)
  print(arr_content,"\n",content_addmat)
  points_mat += content_addmat

  #指示代名詞
  print("---------------指示代名詞------------------------------------------------")
  directive_addmat = Directive_addmat(sentences, target, directive_point)
  points_mat += directive_addmat
  
  #他キーワード
  print("---------------キーワード------------------------------------------------")
  keyword_addmat = Keyword_addmat(sentences, target, df_keywords)
  points_mat += keyword_addmat

  #共通の単語
  print("---------------共通の単語------------------------------------------------")
  commonword_addmat = Commonword_addmat(sentences, target, df_exeptwords, commonwords_coef, commonwords_thres)
  points_mat += commonword_addmat

  #接続詞の加点行列
  print("---------------接続詞------------------------------------------------")
  conjuction_addmat = Paragraph_to_Conjuction_addmat(df_conjuction_attr, df_conjuction_points, sentences, arr_content)
  conjuction_addmat['matrix'] = conjuction_addmat['matrix'].map(lambda x: x + points_mat)
  print("---------------------------------------------------------------------")

  return conjuction_addmat

In [0]:
df_result = Point_matrix("グラフを見ると、バッファサイズが非常に小さいときファイルサイズが大きいと非常に時間がかかっている。またread/writeの方がrecv/sendより僅かの差だが、多くの時間を要している。そして、バッファサイズを大きくしていっても、同様にread/writeがrecv/sendより速くなることはなかった。この要因は、ファイルの通信処理はソケットを用いて通信した方が速くなるからであると考える。",4)

for index, row in df_result.iterrows():
  print(row['label'],row['matrix'],"\n")

---------------要素分類------------------------------------------------
1
1
1
0
['ground', 'ground', 'ground', 'claim'] 
 [[0 5 4 5]
 [5 0 5 6]
 [4 5 0 7]
 [5 6 7 0]]
---------------指示代名詞------------------------------------------------
この  in  3 文目
[[ 0  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0 10]
 [ 0  0 10  0]]
---------------キーワード------------------------------------------------
同様  in  2 文目
[[ 0  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0 10]
 [ 0  0 10  0]]
---------------共通の単語------------------------------------------------
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[0.         0.0952381  0.17391304 0.        ]
 [0.0952381  0.         0.45454545 0.11111111]
 [0.17391304 0.45454545 0.         0.1       ]
 [0.         0.11111111 0.1        0.        ]]
---------------接続詞------------------------------------------------
また  is  ['転換', '対比']
-40 points [ 0 : 1 ] to [ 1 : 6 ] with " 転換 "
array([[  0, -40, -40, -40],
       [-40,   0,   0,   0],
       [-40,   0,   0,   0],
       [-40,   0,   0,   0]

In [0]:
df_result = Point_matrix("図26、図27より、バッファサイズが大きくなる程実行時間が短くなることがわかる.さらに、それぞれの近似曲線より、バッファサイズをx、実行時間をyとすると、両者の関係はy=a/x(aはある特定の定数)という方程式で近似的に表せることができ、実行時間はバッファサイズに反比例していることがわかる.また図27より、read、writeによる実装よりも、fread、fwriteによる実装の方が実行時間が速いことがわかる.実行時間がバッファサイズに反比例したのは、次のような理由が考えられる.cでは、ファイルの内容がバッファサイズごとに読み取り・書き込みが行われる.その処理の回数はバッファサイズに反比例する.よって読み取り・書き込みの回数が増えることによって実行時間が増えると考えられる.read、writeによる実装よりも、fread、fwriteによる実装の方が実行時間が速いのには、以下のような理由が考えられる.cは、両者とも同じバッファサイズで実行しているが、システムコールread、write関数の呼び出し回数が大きく違う.これによって両者の実行時間の違いが出ていると考えられる.",10)

for index, row in df_result.iterrows():
  print(row['label'],row['matrix'],"\n")
  print("--------------------------------------------------------\nthreshold 60")
  for i in range(1, len(row['matrix'])):
    for j in range(i):
      if row['matrix'][i][j] >= 60:
        print(j+1, "→", i+1)
  print("--------------------------------------------------------\nthreshold 55")
  for i in range(1, len(row['matrix'])):
    for j in range(i):
      if row['matrix'][i][j] >= 55:
        print(j+1, "→", i+1)

---------------要素分類------------------------------------------------
1
1
1
1
1
1
0
1
1
1
['ground', 'ground', 'ground', 'ground', 'ground', 'ground', 'claim', 'ground', 'ground', 'claim'] 
 [[ 0  5  4  3  2  1  2 -1 -2 -1]
 [ 5  0  5  4  3  2  3  0 -1  0]
 [ 4  5  0  5  4  3  4  1  0  1]
 [ 3  4  5  0  5  4  5  2  1  2]
 [ 2  3  4  5  0  5  6  3  2  3]
 [ 1  2  3  4  5  0  7  4  3  4]
 [ 2  3  4  5  6  7  0  0 -1  3]
 [-1  0  1  2  3  4  0  0  5  6]
 [-2 -1  0  1  2  3 -1  5  0  7]
 [-1  0  1  2  3  4  3  6  7  0]]
---------------指示代名詞------------------------------------------------
その  in  5 文目
これ  in  9 文目
[[ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0  0  0 10  0]]
---------------キーワード----------------------------------